In [1]:
dados ='/home/dev/PycharmProjects/FallDetection/dados.npy'
rotulos ='/home/dev/PycharmProjects/FallDetection/rotulos.npy'

In [2]:
import numpy as np
X = np.load(dados)
y = np.load(rotulos)

In [3]:
from sklearn.model_selection import train_test_split

# Dividindo os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Dividindo o conjunto de treinamento em conjunto de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



In [4]:
X_train.shape

(988, 400, 1)

In [5]:
X_test.shape

(309, 400, 1)

In [6]:
X_val.shape

(247, 400, 1)

In [7]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

2023-05-30 14:09:11.122106: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 14:09:11.161766: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 14:09:11.162682: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 14:09:11.853703: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,Conv1D,Flatten,MaxPooling1D
import optuna

def objective(trial):
    
    # Definindo o espaço de busca dos hiperparâmetros
    filter_size = trial.suggest_int('filter_size', 8, 256, log = True)
    kernel_size = trial.suggest_int('kernel_size', 2, 7)
    num_layers = trial.suggest_int('num_layers', 2, 6)
    dense_neurons = trial.suggest_int('dense_neurons', 60, 320, log=True)
    max_pool = 2
    dropout = trial.suggest_int('dropout', 2, 5, step=1) / 10.0
 
    model = keras.Sequential()
    
    for i in range(num_layers):
        if i == 0:
            model.add(Conv1D(filters=filter_size, kernel_size=kernel_size, activation="relu", input_shape=(400,1)))
            model.add(MaxPooling1D(pool_size=max_pool))
            model.add(Dropout(dropout))
        else:
            if filter_size < kernel_size:
                filter_size = kernel_size
            filter_size *= 2
            model.add(Conv1D(filters=filter_size, kernel_size=kernel_size, activation="relu"))
            model.add(MaxPooling1D(pool_size=max_pool))
            model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(dense_neurons, activation='relu'))       
    model.add(Dense(8, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        
    historico = model.fit(X_train,y_train,batch_size= 32,epochs = 35, validation_data = (X_val,y_val),verbose =1)
        
    return historico.history['val_accuracy'][-1]



In [9]:
import csv

# criação do objeto de estudo
study = optuna.create_study(direction="maximize")

# abertura do arquivo CSV em modo de escrita
with open("results.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["trial", "accuracy", "hyperparameters"])  # cabeçalho

    # iteração dos trials
    
    study.optimize(objective, n_trials=100)
    best_trial = study.best_trial
    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

    # escrita dos valores no arquivo CSV
    writer.writerow([i, best_trial.value, best_trial.params])

    print("Trial {}: Accuracy = {}".format(i, best_trial.value))

print(study.best_params)

[I 2023-05-30 14:09:12,734] A new study created in memory with name: no-name-16270c53-fc48-49ce-b64d-5ad360aab2e5
2023-05-30 14:09:12.819515: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-30 14:09:12.820156: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/35
31/31 [==============================] - 2s 19ms/step - loss: 2.0097 - accuracy: 0.1923 - val_loss: 2.0579 - val_accuracy: 0.2024
Epoch 2/35
31/31 [==============================] - 0s 13ms/step - loss: 1.9536 - accuracy: 0.2085 - val_loss: 2.0404 - val_accuracy: 0.2024
Epoch 3/35
31/31 [==============================] - 0s 13ms/step - loss: 1.9285 - accuracy: 0.2429 - val_loss: 2.0341 - val_accuracy: 0.2024
Epoch 4/35
31/31 [==============================] - 0s 13ms/step - loss: 1.9107 - accuracy: 0.2460 - val_loss: 2.0280 - val_accuracy: 0.2024
Epoch 5/35
31/31 [==============================] - 0s 13ms/step - loss: 1.8493 - accuracy: 0.3168 - val_loss: 2.0247 - val_accuracy: 0.2024
Epoch 6/35
31/31 [==============================] - 0s 13ms/step - loss: 1.6162 - accuracy: 0.4423 - val_loss: 2.1037 - val_accuracy: 0.2024
Epoch 7/35
31/31 [==============================] - 0s 13ms/step - loss: 1.1431 - accuracy: 0.6326 - val_loss: 2.6313 - val_accuracy: 0.2024
Epoch 8/35
31

[I 2023-05-30 14:09:28,123] Trial 0 finished with value: 0.5060728788375854 and parameters: {'filter_size': 8, 'kernel_size': 3, 'num_layers': 5, 'dense_neurons': 78, 'dropout': 5}. Best is trial 0 with value: 0.5060728788375854.


Epoch 1/35
31/31 [==============================] - 14s 418ms/step - loss: 2.0852 - accuracy: 0.1711 - val_loss: 1.9665 - val_accuracy: 0.1619
Epoch 2/35
31/31 [==============================] - 13s 412ms/step - loss: 1.8379 - accuracy: 0.2713 - val_loss: 1.6154 - val_accuracy: 0.3279
Epoch 3/35
31/31 [==============================] - 12s 401ms/step - loss: 1.3471 - accuracy: 0.5020 - val_loss: 1.1411 - val_accuracy: 0.6113
Epoch 4/35
31/31 [==============================] - 12s 399ms/step - loss: 0.7953 - accuracy: 0.7206 - val_loss: 0.6614 - val_accuracy: 0.7692
Epoch 5/35
31/31 [==============================] - 12s 400ms/step - loss: 0.7175 - accuracy: 0.7652 - val_loss: 0.6348 - val_accuracy: 0.7895
Epoch 6/35
31/31 [==============================] - 13s 406ms/step - loss: 0.5100 - accuracy: 0.8310 - val_loss: 0.3645 - val_accuracy: 0.8462
Epoch 7/35
31/31 [==============================] - 12s 400ms/step - loss: 0.3750 - accuracy: 0.8856 - val_loss: 0.2849 - val_accuracy: 0.8907

[I 2023-05-30 14:16:48,832] Trial 1 finished with value: 0.9433198571205139 and parameters: {'filter_size': 79, 'kernel_size': 7, 'num_layers': 5, 'dense_neurons': 310, 'dropout': 4}. Best is trial 1 with value: 0.9433198571205139.


...

Epoch 1/35
31/31 [==============================] - 154s 5s/step - loss: 2.4601 - accuracy: 0.1984 - val_loss: 1.9666 - val_accuracy: 0.1943
Epoch 2/35
31/31 [==============================] - 153s 5s/step - loss: 1.8240 - accuracy: 0.2743 - val_loss: 1.5715 - val_accuracy: 0.3684
Epoch 3/35
31/31 [==============================] - 153s 5s/step - loss: 1.3533 - accuracy: 0.5283 - val_loss: 1.0396 - val_accuracy: 0.6640
Epoch 4/35
31/31 [==============================] - 153s 5s/step - loss: 1.0140 - accuracy: 0.6700 - val_loss: 0.8291 - val_accuracy: 0.6802
Epoch 5/35
31/31 [==============================] - 153s 5s/step - loss: 0.8709 - accuracy: 0.7014 - val_loss: 0.7242 - val_accuracy: 0.7571
Epoch 6/35
31/31 [==============================] - 153s 5s/step - loss: 0.6037 - accuracy: 0.8067 - val_loss: 0.4746 - val_accuracy: 0.8502
Epoch 7/35
31/31 [==============================] - 153s 5s/step - loss: 0.3925 - accuracy: 0.8765 - val_loss: 0.3296 - val_accuracy: 0.9109
Epoch 8/35
31

[I 2023-05-31 13:50:12,641] Trial 33 finished with value: 0.9514170289039612 and parameters: {'filter_size': 248, 'kernel_size': 6, 'num_layers': 5, 'dense_neurons': 76, 'dropout': 2}. Best is trial 5 with value: 0.9757084846496582.


Epoch 1/35
31/31 [==============================] - 68s 2s/step - loss: 2.3675 - accuracy: 0.1974 - val_loss: 1.9787 - val_accuracy: 0.1943
Epoch 2/35
31/31 [==============================] - 70s 2s/step - loss: 1.8139 - accuracy: 0.2753 - val_loss: 1.4140 - val_accuracy: 0.5466
Epoch 3/35
31/31 [==============================] - 70s 2s/step - loss: 1.0979 - accuracy: 0.6346 - val_loss: 0.7884 - val_accuracy: 0.7045
Epoch 4/35
31/31 [==============================] - 70s 2s/step - loss: 0.6896 - accuracy: 0.7763 - val_loss: 0.6488 - val_accuracy: 0.7611
Epoch 5/35
31/31 [==============================] - 70s 2s/step - loss: 0.5562 - accuracy: 0.8320 - val_loss: 0.4039 - val_accuracy: 0.8623
Epoch 6/35
31/31 [==============================] - 70s 2s/step - loss: 0.3822 - accuracy: 0.8846 - val_loss: 0.8609 - val_accuracy: 0.8421
Epoch 7/35
31/31 [==============================] - 70s 2s/step - loss: 0.3655 - accuracy: 0.8856 - val_loss: 0.3014 - val_accuracy: 0.9069
Epoch 8/35
31/31 [==

[I 2023-05-31 14:31:02,711] Trial 34 finished with value: 0.9109311699867249 and parameters: {'filter_size': 204, 'kernel_size': 4, 'num_layers': 5, 'dense_neurons': 65, 'dropout': 2}. Best is trial 5 with value: 0.9757084846496582.
[W 2023-05-31 14:31:02,943] Trial 35 failed with parameters: {'filter_size': 94, 'kernel_size': 7, 'num_layers': 6, 'dense_neurons': 69, 'dropout': 3} because of the following error: ValueError('Exception encountered when calling layer "conv1d_167" (type Conv1D).\n\nNegative dimension size caused by subtracting 7 from 6 for \'{{node conv1d_167/Conv1D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv1d_167/Conv1D/ExpandDims, conv1d_167/Conv1D/ExpandDims_1)\' with input shapes: [?,1,6,1504], [1,7,1504,3008].\n\nCall arguments received by layer "conv1d_167" (type Conv1D):\n  • inputs=tf.Tensor(shape=(None, 6, 1504), dtype=float32)').
Traceback (most recent

ValueError: Exception encountered when calling layer "conv1d_167" (type Conv1D).

Negative dimension size caused by subtracting 7 from 6 for '{{node conv1d_167/Conv1D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](conv1d_167/Conv1D/ExpandDims, conv1d_167/Conv1D/ExpandDims_1)' with input shapes: [?,1,6,1504], [1,7,1504,3008].

Call arguments received by layer "conv1d_167" (type Conv1D):
  • inputs=tf.Tensor(shape=(None, 6, 1504), dtype=float32)